## 1. Import potrzebnych bibliotek

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (StandardScaler, OneHotEncoder)
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

## 2. Wczytanie bazy danych

In [2]:
dataset = pd.read_csv('Loan_Default.csv', encoding='utf-8')

## 3. Podzielenie bazy danych na zbiór treningowy i testowy

* Musimy podzielić nasz zbiór na zbiór treningowy (80% bazy danych) i zbiór testowy (20%).
* Wybieramy target (y), czyli cechę którą nasz model ma nauczyć się przewidywać. U mnie będzie to Credit_Score.
* Tworzymy zbiór cech (X), czyli bazę danych z wyrzuconą kolumną - tą, którą wybraliśmy na target. 
* Model na podstawie danych z macierzy X będzie dopasowywał najbardziej prawdopodobne wartości w wektorze y.

In [3]:
y = dataset['Status']
X = dataset.drop(['Status'], axis=1)
# axis=1 sprawia, że wyrzucamy kolumny, a nie wiersze
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)
# random_state (pseudo)losowo dzieli bazę danych, można zainicjalizować go jakąś liczbą (jak tutaj), albo ustawić na None (wówczas bazy treningowe i testowe będą inne przy każdej kompliacji)

## 4. Sworzenie data preprocessing pipeline

Data preprocessing pipeline to strumień automatycznie przetwarzający dane, na których potem będziemy trenować model.
Ten krok składa się z czterech części:
1. Wykrycie i obsłużenie wartości brakujących lub outlierów
2. Kodowanie zmiennych kategorycznych
3. Redukcja wielowymiarowości - zminimalizowanie wielkości bazy danych (usunięcie kolumn, które nie wnoszą żadnej nowej informacji)
4. Feature engineering - przetworzenie bazy danych tak, aby jak najlepiej wytrenować na niej model (np. agregacja, transformacja, usuwanie danych)

--- 1. Wykrycie i obsłużenie wartości brakujących lub outlierów

--- 2. Kodowanie zmiennych kategorycznych

In [4]:
# Sprawdzamy, w jakiej kolumnie mamy ile brakujących rekordów
total_rows = len(dataset)
null_report = pd.DataFrame({
    'Null Values': dataset.isnull().sum(),
    'Percent Missing': (round(dataset.isnull().sum() / total_rows, 5)) * 100
})
null_report_sorted = null_report.sort_values(by='Percent Missing', ascending=False)
print(null_report_sorted)

                           Null Values  Percent Missing
Upfront_charges                  39642           26.664
Interest_rate_spread             36639           24.645
rate_of_interest                 36439           24.510
dtir1                            24121           16.225
property_value                   15098           10.155
LTV                              15098           10.155
income                            9150            6.155
loan_limit                        3344            2.249
approv_in_adv                      908            0.611
age                                200            0.135
submission_of_application          200            0.135
loan_purpose                       134            0.090
Neg_ammortization                  121            0.081
term                                41            0.028
year                                 0            0.000
ID                                   0            0.000
business_or_commercial               0          

In [7]:
# Podział kolumn na kategoryczne i numeryczne - aby wiedzieć, w których kolumnach szukać wartości nieprawidłowych i outlierów
categorical_show = [col for col in dataset.columns if dataset[col].dtype == 'object']
numerical_show = [col for col in dataset.columns if dataset[col].dtype in ['int64', 'float64']]
print(f"- Categorical columns: {categorical_show}")
print(f"- Numerical columns: {numerical_show}")

- Categorical columns: ['loan_limit', 'Gender', 'approv_in_adv', 'loan_type', 'loan_purpose', 'Credit_Worthiness', 'open_credit', 'business_or_commercial', 'Neg_ammortization', 'interest_only', 'lump_sum_payment', 'construction_type', 'occupancy_type', 'Secured_by', 'total_units', 'credit_type', 'co-applicant_credit_type', 'age', 'submission_of_application', 'Region', 'Security_Type']
- Numerical columns: ['ID', 'year', 'loan_amount', 'rate_of_interest', 'Interest_rate_spread', 'Upfront_charges', 'term', 'property_value', 'income', 'Credit_Score', 'LTV', 'Status', 'dtir1']


In [5]:
# Szukamy w kolumnach numerycznych wartości nieprawidłowych i outlierów
# Nie rozpatruję kolumn ID oraz year, bo potem i tak je usunę z bazy danych

valid_ranges = {
    'loan_amount': (0, None),
    'rate_of_interest': (0, 100),
    'Upfront_charges': (0, None),
    'term': (0, None),
    'property_value': (0, None),
    'income': (0, None),
    'Credit_Score': (500, 900),
    'LTV': (0, None),
    'dtir1': (0, 100)
}

def analyze_column(col, valid_range):
    invalid_count = 0
    outlier_count = 0
    if col.dtype in ['int64', 'float64']:
        if valid_range:
            if valid_range[1] is not None:
                invalid_count = ((col < valid_range[0]) | (col > valid_range[1])).sum()
            else:
                invalid_count = (col < valid_range[0]).sum()
        Q1 = col.quantile(0.25)
        Q3 = col.quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outlier_count = ((col < lower_bound) | (col > upper_bound)).sum()
    return invalid_count, outlier_count

results = {}
for column in dataset.columns:
    if column in valid_ranges:
        valid_range = valid_ranges[column]
        invalid, outliers = analyze_column(dataset[column], valid_range)
        results[column] = {
            'Invalid Count': invalid,
            'Outlier Count': outliers,
            'Outlier Percent': (outliers/(len(dataset[f'{column}'])))*100
        }
        
results_df = pd.DataFrame(results).T
results_df = results_df.sort_values(by='Outlier Percent', ascending=False)
print(results_df)

                  Invalid Count  Outlier Count  Outlier Percent
term                        0.0        26944.0        18.123360
income                      0.0         6546.0         4.403040
property_value              0.0         5266.0         3.542073
Upfront_charges             0.0         2880.0         1.937176
dtir1                       0.0         2013.0         1.354006
loan_amount                 0.0         1895.0         1.274635
LTV                         0.0         1882.0         1.265891
rate_of_interest            0.0          856.0         0.575772
Credit_Score                0.0            0.0         0.000000


In [6]:
# Dzieliny kolumny na kategoryczne i numeryczne pod pipeline
categorical_columns= [col for col in X_train.columns if X_train[col].dtype == 'object']
numerical_columns = [col for col in X_train.columns if X_train[col].dtype in ['int64', 'float64']]

In [7]:
# Opisujemy, jak postępować z pustymi polami w kolumnach numerycznych i kategorycznych, oraz wprowadzamy One Hot Encoding dla kolumn kategorycznych
# One Hot Encoding = kodowanie cech za pomocą ciągów 0 i 1, np. jeśli baza kolorów to (żółty, czerwony, niebieski), i coś jest żółte, to reprezentuje to wektor (1, 0, 0)
numerical_transformer = Pipeline( steps = [ ('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler()) ] )
# StandardScaler() przetwarza dane w taki sposób, że mają one średnią arytmetyczną 0 oraz odchylenie standardowe 1
categorical_transformer = Pipeline( steps = [ ('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore')) ] )
# handle_unkown='ignore' powoduje, że jeśli w nowej bazie danych pojawi się kolumna, której nie było w zbiorze treningowym, to jest ona ignorowana i nie wyrzuca błędu
pipeline = ColumnTransformer( transformers = [ ('num', numerical_transformer, numerical_columns), ('cat', categorical_transformer, categorical_columns) ] )

In [13]:
# Przetwarzamy zbiór treningowy i testowy
X_train_transformed = pipeline.fit_transform(X_train)
X_test_transformed = pipeline.transform(X_test)
X_train_transformed = pd.DataFrame(X_train_transformed)
X_test_transformed = pd.DataFrame(X_test_transformed)
X_train_transformed.columns = numerical_columns + list(pipeline.named_transformers_['cat']['onehot'].get_feature_names_out())
X_test_transformed.columns = numerical_columns + list(pipeline.named_transformers_['cat']['onehot'].get_feature_names_out())

In [14]:
# Sprawdzamy, czy pipeline działa (1/3)
X_train_transformed.head()

,ID,year,loan_amount,rate_of_interest,Interest_rate_spread,Upfront_charges,term,property_value,income,Credit_Score,...,x17_<25,x17_>74,x18_not_inst,x18_to_inst,x19_North,x19_North-East,x19_central,x19_south,x20_Indriect,x20_direct
0,-0.212857,0.0,-0.133970,-0.575871,-0.497715,0.968485,0.427128,-0.327502,-0.056044,0.121864,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.442086,0.0,-0.842973,-0.084605,-0.085064,-0.164507,0.427128,-0.738286,-0.731314,-0.085258,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
2,1.062304,0.0,-0.842973,-0.084605,-0.085064,-0.164507,0.427128,-0.826311,-0.363887,-0.231969,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
3,1.200252,0.0,-0.788434,-0.084605,1.248028,0.255303,-1.623174,-0.738286,0.142565,0.915831,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
4,-0.972865,0.0,-0.243047,-0.084605,-0.085064,-0.164507,0.427128,-0.210135,-0.622079,1.105692,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0


In [13]:
# Sprawdzamy, czy pipeline działa (2/3)
print(f"Missing values in X_train_transformed: {sum(X_train_transformed.isnull().sum())}")
print(f"Missing values in X_test_transformed: {sum(X_train_transformed.isnull().sum())}")

Missing values in X_train_transformed: 0
Missing values in X_test_transformed: 0


In [14]:
# Zapisujemy zbiór treningowy i testowy

X_train_transformed.to_csv('X_train_transformed.csv', index=False)
X_test_transformed.to_csv('X_test_transformed.csv', index=False)

In [15]:
# Sprawdzamy, czy pipeline działa (3/3)
X_train_transformed.describe(include="all")

,ID,year,loan_amount,rate_of_interest,Interest_rate_spread,Upfront_charges,term,property_value,income,Credit_Score,...,x17_<25,x17_>74,x18_not_inst,x18_to_inst,x19_North,x19_North-East,x19_central,x19_south,x20_Indriect,x20_direct
count,1.189360e+05,118936.0,1.189360e+05,1.189360e+05,1.189360e+05,1.189360e+05,1.189360e+05,1.189360e+05,1.189360e+05,1.189360e+05,...,118936.000000,118936.000000,118936.000000,118936.000000,118936.000000,118936.000000,118936.000000,118936.000000,118936.000000,118936.000000
mean,-3.010977e-17,0.0,-6.499886e-17,-4.471062e-16,-1.393174e-16,2.970353e-16,-3.273840e-16,-1.851990e-17,3.190202e-17,1.061608e-16,...,0.009047,0.048522,0.354266,0.645734,0.502085,0.008526,0.058468,0.430921,0.000219,0.999781
std,1.000004e+00,0.0,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,...,0.094684,0.214867,0.478292,0.478292,0.499998,0.091940,0.234628,0.495207,0.014784,0.014784
min,-1.732802e+00,0.0,-1.715592e+00,-8.251892e+00,-9.126355e+00,-1.092532e+00,-4.083537e+00,-1.413145e+00,-1.138462e+00,-1.724971e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-8.665253e-01,0.0,-7.338957e-01,-5.758705e-01,-5.520171e-01,-6.463474e-01,4.271282e-01,-5.915776e-01,-5.029137e-01,-8.619642e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.895212e-03,0.0,-1.885087e-01,-8.460515e-02,-8.506445e-02,-1.645069e-01,4.271282e-01,-2.101355e-01,-1.851394e-01,-7.587085e-03,...,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,8.653108e-01,0.0,5.750330e-01,4.475990e-01,4.251952e-01,2.968876e-01,4.271282e-01,3.180151e-01,2.319393e-01,8.640502e-01,...,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,1.732024e+00,0.0,1.770018e+01,8.123620e+00,6.569628e+00,1.799911e+01,4.271282e-01,4.700066e+01,6.129425e+01,1.727057e+00,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


--- 3. Redukcja wielowymiarowości

In [16]:
unique_values = X_train_transformed.nunique()
unique_values_text = "\n".join([f"{col} - {count}" for col, count in unique_values.items()])
unique_values_text_sorted = unique_values.sort_values()
print(unique_values_text_sorted)

year                         1
x0_ncf                       2
x1_Female                    2
x1_Joint                     2
x0_cf                        2
                         ...  
income                     937
LTV                       7771
Interest_rate_spread     21524
Upfront_charges          47934
ID                      118936
Length: 71, dtype: int64


In [17]:
# ID jest wszędzie inne, a year wszędzie takie samo, więc te 2 kolumny nie dostarczają nam żadnej nowej informacji i można je wyrzucić
X_train_transformed = X_train_transformed.drop(columns=['year'])
X_test_transformed = X_test_transformed.drop(columns=['year'])
X_train_transformed = X_train_transformed.drop(columns=['ID'])
X_test_transformed = X_test_transformed.drop(columns=['ID'])

In [18]:
print(X_train_transformed.columns)

Index(['loan_amount', 'rate_of_interest', 'Interest_rate_spread',
       'Upfront_charges', 'term', 'property_value', 'income', 'Credit_Score',
       'LTV', 'dtir1', 'x0_cf', 'x0_ncf', 'x1_Female', 'x1_Joint', 'x1_Male',
       'x1_Sex Not Available', 'x2_nopre', 'x2_pre', 'x3_type1', 'x3_type2',
       'x3_type3', 'x4_p1', 'x4_p2', 'x4_p3', 'x4_p4', 'x5_l1', 'x5_l2',
       'x6_nopc', 'x6_opc', 'x7_b/c', 'x7_nob/c', 'x8_neg_amm', 'x8_not_neg',
       'x9_int_only', 'x9_not_int', 'x10_lpsm', 'x10_not_lpsm', 'x11_mh',
       'x11_sb', 'x12_ir', 'x12_pr', 'x12_sr', 'x13_home', 'x13_land',
       'x14_1U', 'x14_2U', 'x14_3U', 'x14_4U', 'x15_CIB', 'x15_CRIF',
       'x15_EQUI', 'x15_EXP', 'x16_CIB', 'x16_EXP', 'x17_25-34', 'x17_35-44',
       'x17_45-54', 'x17_55-64', 'x17_65-74', 'x17_<25', 'x17_>74',
       'x18_not_inst', 'x18_to_inst', 'x19_North', 'x19_North-East',
       'x19_central', 'x19_south', 'x20_Indriect', 'x20_direct'],
      dtype='object')


In [19]:
print(X_test_transformed.columns)

Index(['loan_amount', 'rate_of_interest', 'Interest_rate_spread',
       'Upfront_charges', 'term', 'property_value', 'income', 'Credit_Score',
       'LTV', 'dtir1', 'x0_cf', 'x0_ncf', 'x1_Female', 'x1_Joint', 'x1_Male',
       'x1_Sex Not Available', 'x2_nopre', 'x2_pre', 'x3_type1', 'x3_type2',
       'x3_type3', 'x4_p1', 'x4_p2', 'x4_p3', 'x4_p4', 'x5_l1', 'x5_l2',
       'x6_nopc', 'x6_opc', 'x7_b/c', 'x7_nob/c', 'x8_neg_amm', 'x8_not_neg',
       'x9_int_only', 'x9_not_int', 'x10_lpsm', 'x10_not_lpsm', 'x11_mh',
       'x11_sb', 'x12_ir', 'x12_pr', 'x12_sr', 'x13_home', 'x13_land',
       'x14_1U', 'x14_2U', 'x14_3U', 'x14_4U', 'x15_CIB', 'x15_CRIF',
       'x15_EQUI', 'x15_EXP', 'x16_CIB', 'x16_EXP', 'x17_25-34', 'x17_35-44',
       'x17_45-54', 'x17_55-64', 'x17_65-74', 'x17_<25', 'x17_>74',
       'x18_not_inst', 'x18_to_inst', 'x19_North', 'x19_North-East',
       'x19_central', 'x19_south', 'x20_Indriect', 'x20_direct'],
      dtype='object')
